In [23]:
# -*- coding: utf-8 -*-
'''ResNet50 model for Keras.
# Reference:
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)
Adapted from code contributed by BigMoyan.
'''
from __future__ import print_function

import numpy as np
import warnings
import tensorflow

%env KERAS_BACKEND=tensorflow

import keras
from keras.layers import Input
from keras import layers
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.preprocessing import image
import keras.backend as K
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.engine.topology import get_source_inputs



def identity_block(input_tensor, kernel_size, filters, stage, block):

    filters1, filters2, filters3 = filters
    
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
  

    x = Conv2D(filters1, (1, 1))(input_tensor)
    x = BatchNormalization(axis=bn_axis)(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same')(x)
    x = BatchNormalization(axis=bn_axis)(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1))(x)
    x = BatchNormalization(axis=bn_axis)(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):

    filters1, filters2, filters3 = filters
    
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
        
    x = Conv2D(filters1, (1, 1), strides=strides)(input_tensor)
    x = BatchNormalization(axis=bn_axis)(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same')(x)
    x = BatchNormalization(axis=bn_axis)(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1))(x)
    x = BatchNormalization(axis=bn_axis)(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides)(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis)(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x


def ResNet50(input_tensor=None, input_shape=None,
             pooling=None,
             classes=1000):

    # Determine proper input shape
    input_shape = (32,32,3)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    x = ZeroPadding2D((3, 3))(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
    
    block_shape = K.int_shape(x)

    x = AveragePooling2D(pool_size=(block_shape[1], block_shape[2]),strides=(1, 1))(x)

   
    x = Flatten()(x)
    x = Dense(classes, activation='softmax', name='fc1000')(x)


    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='resnet50')

    return model




   

env: KERAS_BACKEND=tensorflow


In [24]:
 keras.backend.backend()

'tensorflow'

In [25]:
 model = ResNet50(classes = 10)


In [26]:
optimizer = keras.optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [27]:
from keras.datasets import cifar10


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train/255
x_test = x_test/255 

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [28]:
import keras

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [29]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=128),
                        steps_per_epoch=x_train.shape[0] // 128,
                        epochs=100,
                        validation_data=(x_test, y_test))

Epoch 1/100
390/390 [==============================] - 60s - loss: 2.4729 - acc: 0.2055 - val_loss: 2.2552 - val_acc: 0.1932
Epoch 2/100
390/390 [==============================] - 49s - loss: 1.8500 - acc: 0.3340 - val_loss: 2.1240 - val_acc: 0.3474
Epoch 3/100
390/390 [==============================] - 49s - loss: 1.5394 - acc: 0.4431 - val_loss: 1.5906 - val_acc: 0.4459
Epoch 4/100
390/390 [==============================] - 49s - loss: 1.3877 - acc: 0.4984 - val_loss: 1.3233 - val_acc: 0.5320
Epoch 5/100
390/390 [==============================] - 49s - loss: 1.2775 - acc: 0.5422 - val_loss: 1.3034 - val_acc: 0.5491
Epoch 6/100
390/390 [==============================] - 49s - loss: 1.1913 - acc: 0.5759 - val_loss: 1.6572 - val_acc: 0.4607
Epoch 7/100
390/390 [==============================] - 49s - loss: 1.1166 - acc: 0.6019 - val_loss: 1.1349 - val_acc: 0.6028
Epoch 8/100
390/390 [==============================] - 49s - loss: 1.0496 - acc: 0.6270 - val_loss: 1.0502 - val_acc: 0.6298


390/390 [==============================] - 49s - loss: 0.2021 - acc: 0.9289 - val_loss: 0.9889 - val_acc: 0.7786
Epoch 67/100
390/390 [==============================] - 49s - loss: 0.2025 - acc: 0.9295 - val_loss: 0.8573 - val_acc: 0.7782
Epoch 68/100
390/390 [==============================] - 49s - loss: 0.1933 - acc: 0.9307 - val_loss: 0.9279 - val_acc: 0.7858
Epoch 69/100
390/390 [==============================] - 49s - loss: 0.1931 - acc: 0.9337 - val_loss: 1.1510 - val_acc: 0.7429
Epoch 70/100
390/390 [==============================] - 49s - loss: 0.1925 - acc: 0.9328 - val_loss: 0.9661 - val_acc: 0.7652
Epoch 71/100
390/390 [==============================] - 49s - loss: 0.1825 - acc: 0.9374 - val_loss: 1.0358 - val_acc: 0.7599
Epoch 72/100
390/390 [==============================] - 49s - loss: 0.1830 - acc: 0.9369 - val_loss: 0.9303 - val_acc: 0.7828
Epoch 73/100
390/390 [==============================] - 49s - loss: 0.1796 - acc: 0.9377 - val_loss: 0.9865 - val_acc: 0.7766
Epoch

In [3]:
import keras
print(keras.__version__)

2.0.2


In [12]:
from keras import backend

In [15]:
 keras.backend.backend()

'theano'

In [7]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.